In [130]:
import pandas as pd
import pandasql as ps
import numpy as np
import warnings

In [131]:
# 1. Most Underrated Team in Vegas over the last 10 seasons against the spread
# 2. Does playing in a dome have an impact on total score?
#     - During the winter months does Dome play increase overall score compared to bad outdoor weather conditions
# 3. Trajectory of average points scored in NFL by year from 


# Pass NFL Betting data subset into a dataframe
df = pd.read_csv('NFL_Bet_Data.csv')
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [132]:
# NFL team that has covered the most spreads over the last 10 seasons - AKA Most Undervalued betting team to Vegas
# Drop any line of data that doesn't have spread_favorite and remove any data that is less than 2012 season
# Remove playoff games
# Remove columns that won't be used
spread_df = df[~df['spread_favorite'].isnull()]
spread_df = spread_df.loc[spread_df['schedule_season'] > 2011]
spread_df = spread_df.loc[(spread_df['schedule_playoff'] == False)]
spread_df = spread_df[['schedule_season', 'schedule_week', 'team_home', 'team_away', 'score_home', 'score_away', 'team_favorite_id', 'spread_favorite']]
spread_df = spread_df.reset_index()
spread_df

,index,schedule_season,schedule_week,team_home,team_away,score_home,score_away,team_favorite_id,spread_favorite
0,10542,2012,1,New York Giants,Dallas Cowboys,17.0,24.0,NYG,-4.0
1,10543,2012,1,Arizona Cardinals,Seattle Seahawks,20.0,16.0,SEA,-3.0
2,10544,2012,1,Chicago Bears,Indianapolis Colts,41.0,21.0,CHI,-10.0
3,10545,2012,1,Cleveland Browns,Philadelphia Eagles,16.0,17.0,PHI,-9.5
4,10546,2012,1,Denver Broncos,Pittsburgh Steelers,31.0,19.0,DEN,-2.0
...,...,...,...,...,...,...,...,...,...
2665,13321,2022,6,New York Giants,Baltimore Ravens,24.0,20.0,BAL,-5.5
2666,13322,2022,6,Philadelphia Eagles,Dallas Cowboys,26.0,17.0,PHI,-6.5
2667,13323,2022,6,Pittsburgh Steelers,Tampa Bay Buccaneers,20.0,18.0,TB,-10.0
2668,13324,2022,6,Seattle Seahawks,Arizona Cardinals,19.0,9.0,ARI,-2.5


In [133]:
# Alter team names that have changed over the last 10 years to the current team name

spread_df.loc[spread_df['team_home'] == 'San Diego Chargers', 'team_home'] = 'Los Angeles Chargers'
spread_df.loc[spread_df['team_home'] == 'St. Louis Rams', 'team_home'] = 'Los Angeles Rams'
spread_df.loc[spread_df['team_home'] == 'Oakland Raiders', 'team_home'] = 'Las Vegas Raiders'
spread_df.loc[spread_df['team_home'] == 'Washington Football Team', 'team_home'] = 'Washington Commanders'
spread_df.loc[spread_df['team_home'] == 'Washington Redskins', 'team_home'] = 'Washington Commanders'

spread_df.loc[spread_df['team_away'] == 'San Diego Chargers', 'team_away'] = 'Los Angeles Chargers'
spread_df.loc[spread_df['team_away'] == 'St. Louis Rams', 'team_away'] = 'Los Angeles Rams'
spread_df.loc[spread_df['team_away'] == 'Oakland Raiders', 'team_away'] = 'Las Vegas Raiders'
spread_df.loc[spread_df['team_away'] == 'Washington Football Team', 'team_away'] = 'Washington Commanders'
spread_df.loc[spread_df['team_away'] == 'Washington Redskins', 'team_away'] = 'Washington Commanders'

print(pd.concat([spread_df['team_home'], spread_df['team_away']]).unique())

['New York Giants' 'Arizona Cardinals' 'Chicago Bears' 'Cleveland Browns'
 'Denver Broncos' 'Detroit Lions' 'Green Bay Packers' 'Houston Texans'
 'Kansas City Chiefs' 'Minnesota Vikings' 'New Orleans Saints'
 'New York Jets' 'Tampa Bay Buccaneers' 'Tennessee Titans'
 'Baltimore Ravens' 'Las Vegas Raiders' 'Buffalo Bills'
 'Carolina Panthers' 'Cincinnati Bengals' 'Indianapolis Colts'
 'Jacksonville Jaguars' 'Miami Dolphins' 'New England Patriots'
 'Philadelphia Eagles' 'Pittsburgh Steelers' 'Los Angeles Chargers'
 'San Francisco 49ers' 'Seattle Seahawks' 'Los Angeles Rams'
 'Atlanta Falcons' 'Dallas Cowboys' 'Washington Commanders']


In [134]:
# Bring in a CSV breaking down team abbreviation so I can equate favorite to an actual team name
# PICK means that there isn't a favorite and teams are equal
team_df = pd.read_csv("TeamABV.csv")
team_df

,Abbreviation,Team_Favorite
0,NYG,New York Giants
1,SEA,Seattle Seahawks
2,CHI,Chicago Bears
3,PHI,Philadelphia Eagles
4,DEN,Denver Broncos
5,DET,Detroit Lions
6,GB,Green Bay Packers
7,HOU,Houston Texans
8,ATL,Atlanta Falcons
9,MIN,Minnesota Vikings


In [135]:
# Merge the 2 dataframes together and then drop the abbreviation/index/team_favorite_id fields
spread_df = ps.sqldf("Select * FROM spread_df sdf join team_df tdf on sdf.team_favorite_id	 = tdf.Abbreviation")
spread_df = spread_df.drop(columns=['index', 'team_favorite_id', 'Abbreviation'])
spread_df

,schedule_season,schedule_week,team_home,team_away,score_home,score_away,spread_favorite,Team_Favorite
0,2012,1,New York Giants,Dallas Cowboys,17.0,24.0,-4.0,New York Giants
1,2012,1,Arizona Cardinals,Seattle Seahawks,20.0,16.0,-3.0,Seattle Seahawks
2,2012,1,Chicago Bears,Indianapolis Colts,41.0,21.0,-10.0,Chicago Bears
3,2012,1,Cleveland Browns,Philadelphia Eagles,16.0,17.0,-9.5,Philadelphia Eagles
4,2012,1,Denver Broncos,Pittsburgh Steelers,31.0,19.0,-2.0,Denver Broncos
...,...,...,...,...,...,...,...,...
2665,2022,6,New York Giants,Baltimore Ravens,24.0,20.0,-5.5,Baltimore Ravens
2666,2022,6,Philadelphia Eagles,Dallas Cowboys,26.0,17.0,-6.5,Philadelphia Eagles
2667,2022,6,Pittsburgh Steelers,Tampa Bay Buccaneers,20.0,18.0,-10.0,Tampa Bay Buccaneers
2668,2022,6,Seattle Seahawks,Arizona Cardinals,19.0,9.0,-2.5,Arizona Cardinals


In [136]:
# Adjust the home team scored based on the favored team and the point spread
# If the home team is favored, subtract the point spread. If the home team is the underdog, add the point spread

spread_df['home_team_adjusted'] = np.where(spread_df['team_home'] == spread_df['Team_Favorite'], spread_df['score_home'] + spread_df['spread_favorite'],
                               np.where(spread_df['team_home'] != spread_df['Team_Favorite'], spread_df['score_home'] - spread_df['spread_favorite'], "Wrong"))

# Convert home team adjusted score to a float
spread_df['home_team_adjusted'] = spread_df['home_team_adjusted'].astype(float)

# Compared adjusted home team score to the away team score to see who won against the point spread, return winning team
# If the adjusted and away team score are equal, return Push
# Then drop the home_team_adjusted score field

spread_df['Spread_Winner'] = np.where(spread_df['home_team_adjusted'] > spread_df['score_away'], spread_df['team_home'],
                           np.where(spread_df['score_away'] > spread_df['home_team_adjusted'], spread_df['team_away'], 'Push'))
spread_df = spread_df.drop(columns=['home_team_adjusted'])
spread_df

# Output a CSV file to be used in Tableau and build visualization
spread_df.to_csv('spread_data.csv')

In [137]:
# Most Favored Team over last 10 seasons
Most_fav = ps.sqldf("Select Team_Favorite, count(Team_Favorite) Total_Times_Favored from spread_df group by Team_Favorite order by Total_Times_Favored desc")
Most_fav

,Team_Favorite,Total_Times_Favored
0,New England Patriots,137
1,Green Bay Packers,122
2,Seattle Seahawks,120
3,Kansas City Chiefs,115
4,Baltimore Ravens,112
5,New Orleans Saints,109
6,Pittsburgh Steelers,106
7,Denver Broncos,99
8,Philadelphia Eagles,98
9,Dallas Cowboys,96


In [139]:
# Does playing in a dome have an impact on Total Points Scored
# Is there a larger delta between Dome and Non-Dome during the winter months (November, December, January)
total_df = df[['schedule_date', 'schedule_season', 'schedule_week', 'team_home', 'team_away', 'score_home', 'score_away', 'stadium', 'weather_detail']]
total_df   

,schedule_date,schedule_season,schedule_week,team_home,team_away,score_home,score_away,stadium,weather_detail
0,9/2/1966,1966,1,Miami Dolphins,Oakland Raiders,14.0,23.0,Orange Bowl,NaN
1,9/3/1966,1966,1,Houston Oilers,Denver Broncos,45.0,7.0,Rice Stadium,NaN
2,9/4/1966,1966,1,San Diego Chargers,Buffalo Bills,27.0,7.0,Balboa Stadium,NaN
3,9/9/1966,1966,2,Miami Dolphins,New York Jets,14.0,19.0,Orange Bowl,NaN
4,9/10/1966,1966,1,Green Bay Packers,Baltimore Colts,24.0,3.0,Lambeau Field,NaN
...,...,...,...,...,...,...,...,...,...
13499,1/8/2023,2022,18,Philadelphia Eagles,New York Giants,NaN,NaN,Lincoln Financial Field,NaN
13500,1/8/2023,2022,18,Pittsburgh Steelers,Cleveland Browns,NaN,NaN,Acrisure Stadium,NaN
13501,1/8/2023,2022,18,San Francisco 49ers,Arizona Cardinals,NaN,NaN,Levi's Stadium,NaN
13502,1/8/2023,2022,18,Seattle Seahawks,Los Angeles Rams,NaN,NaN,Lumen Field,NaN
